# Sector gating: keep sectors 82+83

This step creates a gated light curve that excludes sectors with clear depth inconsistencies.
We use this gated dataset as the baseline for re-running downstream checks (V11–V21, V16–V19) to see which warnings persist.


In [ ]:
from pathlib import Path
import json
import sys

import numpy as np

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh
import tess_vetter.api as btv

ds = sh.load_dataset()
sectors_used = [82, 83]

rows = []
for s in sectors_used:
    lc_s = ds.lc_by_sector[s]
    q = lc_s.quality if getattr(lc_s, 'quality', None) is not None else np.zeros(len(lc_s.time))
    rows.append({
        'time': np.asarray(lc_s.time, dtype=np.float64),
        'flux': np.asarray(lc_s.flux, dtype=np.float64),
        'flux_err': np.asarray(lc_s.flux_err, dtype=np.float64),
        'sector': int(s),
        'quality': np.asarray(q, dtype=np.int32),
    })

stitched = btv.stitch_lightcurves(rows)
lc = btv.LightCurve(time=stitched.time, flux=stitched.flux, flux_err=stitched.flux_err, quality=stitched.quality)

depth_ppm, depth_err_ppm = sh.estimate_depth_ppm(lc)

print(json.dumps({
  'sectors_used': sectors_used,
  'depth_ppm': depth_ppm,
  'depth_err_ppm': depth_err_ppm,
  'n_points': int(len(lc.time)),
}, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "depth_err_ppm": 11.153641839855947,
  "depth_ppm": 264.57118442002604,
  "n_points": 35459,
  "sectors_used": [
    82,
    83
  ]
}
```

</details>


In [ ]:
import json
import matplotlib.pyplot as plt

from tess_vetter.plotting import plot_full_lightcurve, plot_phase_folded

candidate = sh.make_candidate(depth_ppm)

run_out_dir, docs_out_dir = sh.artifact_dirs(step_id='24_sector_gating_82_83')
full_name = 'gated_full_lightcurve.png'
fold_name = 'gated_phase_folded.png'

# Full light curve
fig, ax = plt.subplots(figsize=(10, 4))
plot_full_lightcurve(lc, ax=ax, candidate=candidate)
ax.set_title('Gated sectors 82+83: full light curve (PDCSAP)')
fig.tight_layout()
fig.savefig(run_out_dir / full_name, dpi=160, bbox_inches='tight')
if docs_out_dir is not None:
    fig.savefig(docs_out_dir / full_name, dpi=160, bbox_inches='tight')
plt.show()
plt.close(fig)

# Phase-folded transit
fig, ax = plt.subplots(figsize=(8, 5))
plot_phase_folded(lc, candidate, ax=ax, bin_minutes=30.0, phase_range=(-0.25, 0.25))
ax.set_title('Gated sectors 82+83: phase-folded transit')
fig.tight_layout()
fig.savefig(run_out_dir / fold_name, dpi=160, bbox_inches='tight')
if docs_out_dir is not None:
    fig.savefig(docs_out_dir / fold_name, dpi=160, bbox_inches='tight')
plt.show()
plt.close(fig)

print(json.dumps({
  'status': 'ok',
  'run_plot_path_full': str(run_out_dir / full_name),
  'docs_plot_path_full': str((docs_out_dir / full_name) if docs_out_dir is not None else None),
  'run_plot_path_folded': str(run_out_dir / fold_name),
  'docs_plot_path_folded': str((docs_out_dir / fold_name) if docs_out_dir is not None else None)
}, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path_folded": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/24_sector_gating_82_83/gated_phase_folded.png",
  "docs_plot_path_full": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/24_sector_gating_82_83/gated_full_lightcurve.png",
  "run_plot_path_folded": "persistent_cache/tutorial_toi-5807-incremental/24_sector_gating_82_83/gated_phase_folded.png",
  "run_plot_path_full": "persistent_cache/tutorial_toi-5807-incremental/24_sector_gating_82_83/gated_full_lightcurve.png",
  "status": "ok"
}
```

</details>


## Plots

<img src="../artifacts/tutorial_toi-5807-incremental/24_sector_gating_82_83/gated_full_lightcurve.png" width="920" />

<img src="../artifacts/tutorial_toi-5807-incremental/24_sector_gating_82_83/gated_phase_folded.png" width="820" />


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none (this is a preprocessing step).
- **Result:** built a stitched light curve from sectors 82+83 only; depth estimate updates accordingly.
- **Why this is useful:** downstream checks are only interpretable if the input dataset is internally consistent; gating removes known-problem sectors before re-evaluating diagnostics.
- **Interpretation:** this gated dataset is a working baseline for re-running checks; it is not a final decision about which sectors are “correct”.
- **Next step:** re-run the key warning-producing diagnostics (start with V16–V19) on the gated dataset and compare flags/metrics to the all-sector run.

</details>
